In [1]:
import pandas as pd

df_clients = pd.read_csv('./content/caso2/clients.csv')
df_depots = pd.read_csv('./content/caso2/depots.csv')
df_vehicles = pd.read_csv('./content/caso2/vehicles.csv')

In [2]:
df_depots['DepotID'] = df_depots['DepotID'].astype(str)
df_depots['ClientID'] = df_clients['ClientID'].astype(str)
df_vehicles['VehicleName'] = [f'V{i+1}' for i in range(len(df_vehicles))]


coord_depots = list(zip(df_depots['Longitude'], df_depots['Latitude']))

coord_clients = list(zip(df_clients['Longitude'], df_clients['Latitude']))

lugares = coord_depots + coord_clients

In [3]:
from pyomo.environ import *
import requests

model = ConcreteModel()

# Definición de conjuntos
model.D = Set(initialize=df_depots['DepotID'].apply(lambda x: f'CD{x}'))
model.C = Set(initialize=df_clients['ClientID'].apply(lambda x: f'C{x}'))
model.V = Set(initialize=df_vehicles['VehicleName'].tolist())
model.N = model.D | model.C

In [4]:
import openrouteservice
from openrouteservice import convert
import json

API_KEY = '5b3ce3597851110001cf6248781c341d9b2c47b4b5f6ce22fb428092'

client = openrouteservice.Client(key=API_KEY)

matrix = client.distance_matrix(
    locations=lugares,
    profile='driving-car',
    metrics=['distance'],
    resolve_locations=True,
    units='km',
)

In [5]:
precio_km = 3032.1
cost = {}
distance = {}

dist_matrix = matrix['distances']


for i,i1 in zip(range(len(lugares)),  list(model.N.data())):
        for j,j1 in zip(range(len(lugares)),  list(model.N.data())):
            if i != j:
                distancia_km = dist_matrix[i][j]
                precio = round(distancia_km * precio_km, 3)
            else:
                distancia_km = 9999999
                precio =9999999
            distance[(i1, j1)] = distancia_km
            cost[(i1, j1)] = precio

In [6]:
depot_capacity = {}
for depot, d in zip(df_depots['DepotID'], df_depots['Capacity']):
    depot_capacity[f'CD{depot}'] = d


demand = {}
for client, c in zip(df_clients['ClientID'], df_clients['Demand']):
    demand[f'C{client}'] = c


vehicle_capacity={}
for vehicle, c in zip(df_vehicles['VehicleName'], df_vehicles['Capacity']):
    vehicle_capacity[vehicle] = c


vehicle_range={}
for vehicle, r in zip(df_vehicles['VehicleName'], df_vehicles['Range']):
    vehicle_range[vehicle] = r


In [7]:
model.cost = Param(model.N, model.N, initialize=lambda model, i, j: cost[(i, j)], within=NonNegativeReals)
model.distance = Param(model.N, model.N, initialize=lambda model, i, j: distance[(i, j)], within=NonNegativeReals)

# Variables de decisión y auxiliares

model.x = Var(model.V, model.N, model.N, domain=Binary)
model.y = Var(model.V, model.D, domain=Binary)
model.u = Var(model.V, model.C, domain=NonNegativeReals)

# funcion objetivo

def obj_rule(model):
    return sum(model.cost[i,j] * model.x[v,i,j] for v in model.V for i in model.N for j in model.N if i != j)

model.obj = Objective(rule=obj_rule, sense=minimize)

# Restricciones

def assign_depot_rule(model, v):
    return sum(model.y[v,d] for d in model.D) == 1

model.assign_depot = Constraint(model.V, rule=assign_depot_rule)

def start_route_rule(model, v, d):
    return sum(model.x[v, d, j] for j in model.N if j != d) == model.y[v,d]

model.start_route = Constraint(model.V, model.D, rule=start_route_rule)


def flow_conservation_rule(model, v, i):
    if i in model.C:
        return sum(model.x[v, j, i] for j in model.N if j != i) == \
               sum(model.x[v, i, j] for j in model.N if j != i)
    else:
        return Constraint.Skip

model.flow_conservation = Constraint(model.V, model.N, rule=flow_conservation_rule)

def no_depot_to_depot_rule(model, v, i, j):
    if i in model.D and j in model.D and i != j:
        return model.x[v,i,j] == 0
    else:
        return Constraint.Skip

model.no_depot_travel = Constraint(model.V, model.N, model.N, rule=no_depot_to_depot_rule)

def customer_visit_rule(model, i):
    if i in model.C:
        return sum(model.x[v, j, i] for v in model.V for j in model.N if j != i) == 1
    else:
        return Constraint.Skip

model.customer_visit = Constraint(model.N, rule=customer_visit_rule)

def vehicle_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= vehicle_capacity[v]

model.vehicle_capacity = Constraint(model.V, rule=vehicle_capacity_rule)

def depot_capacity_rule(model, v):
    return sum(demand[i] * sum(model.x[v, j, i] for j in model.N if j != i) for i in model.C) <= sum(model.y[v,d] * depot_capacity[d] for d in model.D)

model.depot_capacity = Constraint(model.V, rule=depot_capacity_rule)

def vehicle_range_rule(model, v):
    return sum(model.distance[i,j] * model.x[v,i,j] for i in model.N for j in model.N if i != j) <= vehicle_range[v]

model.vehicle_range = Constraint(model.V, rule=vehicle_range_rule)

def mtz_rule(model, v, i, j):
    if i != j:
        return model.u[v,i] - model.u[v,j] + vehicle_capacity[v] * model.x[v,i,j] <= vehicle_capacity[v] - demand[j]
    return Constraint.Skip

model.mtz = Constraint(model.V, model.C, model.C, rule=mtz_rule)

def mtz_bounds_rule(model, v, i):
    return (demand[i], model.u[v,i], vehicle_capacity[v])

model.mtz_bounds = Constraint(model.V, model.C, rule=mtz_bounds_rule)

def demand_satisfaction_rule(model, v, i):
    return model.u[v,i] >= demand[i] * sum(model.x[v, j, i] for j in model.N if j != i)

model.demand_satisfaction = Constraint(model.V, model.C, rule=demand_satisfaction_rule)

from amplpy import modules

# resolver
solver_name = "gurobi"
solver = SolverFactory(solver_name+"nl", executable=modules.find(solver_name), solve_io="nl")
results = solver.solve(model)

In [14]:
import csv
import os
from pyomo.environ import value

def print_and_save_routes(model, output_file="solutions/verificacion_caso2.csv"):
    # Crear la carpeta 'solutions' si no existe
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Abrir el archivo CSV para escribir
    with open(output_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        
        # Escribir la cabecera del CSV
        writer.writerow([
            "VehicleId", "DepotId", "InitialLoad", "RouteSequence", 
            "ClientsServed", "DemandsSatisfied", "TotalDistance", "TotalTime", "FuelCost"
        ])
        
        # Imprimir el costo total
        print(f"\n==== Costo óptimo total del plan de rutas: $ {value(model.obj):.3f}")
        
        # Recorrer cada vehículo
        for v in model.V:
            print(f"\n>>> Ruta del vehículo {v}:")

            # Encontrar el depósito asignado y su capacidad
            depot = None
            for d in model.D:
                if value(model.y[v, d]) > 0.5:
                    depot = d
                    break
            
            if depot is None:
                print("Vehículo no asignado a ningún depósito.")
                continue
            
            # Inicializar variables para la ruta
            current_node = depot
            route = [current_node]
            visited = set()
            deliveries = {}
            total_load = 0
            total_distance = 0
            
            # Reconstruir la ruta del vehículo
            while True:
                next_node = None
                for j in model.N:
                    if j != current_node and value(model.x[v, current_node, j]) > 0.5:
                        next_node = j
                        break
                
                if next_node is None or next_node in visited:
                    break
                
                # Si es un cliente, actualizar la carga y las entregas
                if next_node in model.C:
                    deliveries[(current_node, next_node)] = demand[next_node]
                    total_load += demand[next_node]
                
                route.append(next_node)
                visited.add(next_node)
                current_node = next_node

                # Calcular la distancia total recorrida
                total_distance += value(model.distance[route[-2], route[-1]])

            # Calcular el uso del rango operativo
            vehicle_range_val = vehicle_range[v]
            porcentaje = (total_distance / vehicle_range_val) * 100 if vehicle_range_val > 0 else 0

            # Crear la secuencia de clientes atendidos
            clients_served = len(deliveries)
            demands_satisfied = '-'.join(str(demand[i]) for i in route if i in model.C)

            # Crear la secuencia de rutas (como una cadena separada por guiones)
            route_sequence = '-'.join(route)

            # Calcular el costo de combustible (distancia * precio por km)
            fuel_cost = total_distance * precio_km

            # Imprimir los resultados
            if len(route) > 1:
                print(f" - Ruta: {' → '.join(route)}")
                print(f" - Carga total transportada: {total_load}/{vehicle_capacity[v]} unidades")
                print(f" - Distancia total recorrida: {total_distance:.3f} km")
                print(f" - Uso del rango operativo: {porcentaje:.1f}%")
                print(f' - Costo del transporte: $ {fuel_cost:.2f}')

                print(" - Entregas entre nodos:")
                for (i, j), qty in deliveries.items():
                    print(f"    De {i} → {j}: {qty} unidades")
            else:
                print("Vehículo no utilizado.")

            # Guardar los resultados en el CSV
            writer.writerow([
                f"VEH{v[1:]}", depot, 
                total_load, route_sequence, 
                clients_served, demands_satisfied, 
                total_distance, 0, fuel_cost
            ])

# Llamar la función para imprimir y guardar las rutas
print_and_save_routes(model)



==== Costo óptimo total del plan de rutas: $ 284744.511

>>> Ruta del vehículo V1:
 - Ruta: CD9 → C5 → CD9
 - Carga total transportada: 5/131.9211396722696 unidades
 - Distancia total recorrida: 17.540 km
 - Uso del rango operativo: 12.0%
 - Costo del transporte: $ 53183.03
 - Entregas entre nodos:
    De CD9 → C5: 5 unidades

>>> Ruta del vehículo V2:
 - Ruta: CD9 → C8 → C7 → C9 → CD7
 - Carga total transportada: 37/108.4356199315333 unidades
 - Distancia total recorrida: 30.460 km
 - Uso del rango operativo: 2.3%
 - Costo del transporte: $ 92357.77
 - Entregas entre nodos:
    De CD9 → C8: 10 unidades
    De C8 → C7: 12 unidades
    De C7 → C9: 15 unidades

>>> Ruta del vehículo V3:
 - Ruta: CD5 → C4 → C1 → CD5
 - Carga total transportada: 18/91.50425520531196 unidades
 - Distancia total recorrida: 22.730 km
 - Uso del rango operativo: 2.4%
 - Costo del transporte: $ 68919.63
 - Entregas entre nodos:
    De CD5 → C4: 6 unidades
    De C4 → C1: 12 unidades

>>> Ruta del vehículo V4:


In [9]:
import folium
import openrouteservice
from openrouteservice import convert
from pyomo.environ import value
import itertools

# Paleta de colores (puedes expandirla si tienes más vehículos)
colores = ['red', 'blue', 'green', 'orange', 'purple', 'brown', 'black', 'pink', 'gray', 'cyan']
vehiculo_a_color = dict(zip(model.V, itertools.cycle(colores)))


client = openrouteservice.Client(key=API_KEY)

m = folium.Map(location=[4.65, -74.1], zoom_start=11)

coords= lugares
labels= list(model.N.data())

In [10]:
for label, coord in zip(labels, coords):
    folium.Marker(location=(coord[1], coord[0]), popup=label).add_to(m)

# Diccionario de índice a coordenada
label_to_coord = dict(zip(labels, coords))

# Simulación: rutas como lista de tuplas (vehículo, origen, destino)
rutas_optimas = []
for v in model.V:
    for i in model.N:
        for j in model.N:
            if i != j and value(model.x[v,i,j]) > 0.5:
                rutas_optimas.append((v, i, j))

In [11]:
for (v, i, j) in rutas_optimas:
    coord_i = label_to_coord[i]
    coord_j = label_to_coord[j]
    color = vehiculo_a_color[v]
    try:
        route = client.directions(
            coordinates=[coord_i, coord_j],
            profile='driving-car',
            format='geojson'
        )
        folium.GeoJson(
            route,
            name=f"Ruta {v}: {i}->{j}",
            tooltip=f"{v}: {i} → {j}",
            style_function=lambda x, col=color: {'color': col, 'weight': 4, 'opacity': 0.8}
        ).add_to(m)
    except Exception as e:
        print(f"Error en ruta {i} → {j}: {e}")
m